In [6]:
# Install required packages
!pip install pyspark
!pip install plotly
!pip install numpy

# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, min, max, count, stddev, when, sum as spark_sum, round
import random
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Initialize Spark session
spark = SparkSession.builder \
    .appName("ResultManagementSystem") \
    .getOrCreate()

# Define subject-specific parameters for mark generation
subject_params = {
    "Electronics": {"pass_rate": 0.7, "max": 90},
    "Programming": {"pass_rate": 0.8, "max": 95},
    "Database": {"pass_rate": 0.6, "max": 85},
    "Data Science": {"pass_rate": 0.55, "max": 80},
    "Mathematics": {"pass_rate": 0.5, "max": 75},
    "DSA": {"pass_rate": 0.75, "max": 92}
}

# Define departments
departments = ["CSE", "EEE", "ECE", "MECH", "CIVIL"]

# Generate student profiles and subject marks
def generate_student_profile():
    subjects = ["Electronics", "Programming", "Database", "Data Science", "Mathematics", "DSA"]

    student_id = random.randint(1, 10000)
    name = f"Student_{student_id}"
    department = random.choice(departments)

    marks = {}
    for subject in subjects:
        params = subject_params[subject]
        # Generate marks based on desired pass rate
        if random.random() < params["pass_rate"]:
            # If passing, generate a mark between 40 and the maximum
            mark = random.randint(40, params["max"])
        else:
            # If failing, generate a mark between 0 and 39
            mark = random.randint(0, 39)

        marks[subject] = mark

    return (student_id, name, department, marks)

# Generate data for 10,000 students
students_data = [generate_student_profile() for _ in range(10000)]

# Create a Spark DataFrame
df = spark.createDataFrame(students_data, ["Student_ID", "Name", "Department", "marks"])

# Explode the marks column to create a flat structure
df = df.select("Student_ID", "Name", "Department",
               col("marks.Electronics").alias("Electronics"),
               col("marks.Programming").alias("Programming"),
               col("marks.Database").alias("Database"),
               col("marks.Data Science").alias("Data_Science"),
               col("marks.Mathematics").alias("Mathematics"),
               col("marks.DSA").alias("DSA"))

# Calculate pass/fail status for each subject
for subject in ["Electronics", "Programming", "Database", "Data_Science", "Mathematics", "DSA"]:
    df = df.withColumn(f"{subject}_Status", when(col(subject) >= 40, "Pass").otherwise("Fail"))

# Calculate average total marks
df = df.withColumn("Average_Total_Marks",
                   round((col("Electronics") + col("Programming") + col("Database") +
                          col("Data_Science") + col("Mathematics") + col("DSA")) / 6, 2))

# Perform basic analysis
analysis = df.select(
    avg("Electronics").alias("avg_electronics"),
    avg("Programming").alias("avg_programming"),
    avg("Database").alias("avg_database"),
    avg("Data_Science").alias("avg_data_science"),
    avg("Mathematics").alias("avg_mathematics"),
    avg("DSA").alias("avg_dsa"),
    min("Electronics").alias("min_electronics"),
    min("Programming").alias("min_programming"),
    min("Database").alias("min_database"),
    min("Data_Science").alias("min_data_science"),
    min("Mathematics").alias("min_mathematics"),
    min("DSA").alias("min_dsa"),
    max("Electronics").alias("max_electronics"),
    max("Programming").alias("max_programming"),
    max("Database").alias("max_database"),
    max("Data_Science").alias("max_data_science"),
    max("Mathematics").alias("max_mathematics"),
    max("DSA").alias("max_dsa"),
    stddev("Electronics").alias("std_electronics"),
    stddev("Programming").alias("std_programming"),
    stddev("Database").alias("std_database"),
    stddev("Data_Science").alias("std_data_science"),
    stddev("Mathematics").alias("std_mathematics"),
    stddev("DSA").alias("std_dsa")
).collect()[0]

# Calculate pass/fail statistics
pass_fail_stats = df.select(
    count("*").alias("total_students"),
    count(col("Electronics") >= 40).alias("pass_electronics"),
    count(col("Programming") >= 40).alias("pass_programming"),
    count(col("Database") >= 40).alias("pass_database"),
    count(col("Data_Science") >= 40).alias("pass_data_science"),
    count(col("Mathematics") >= 40).alias("pass_mathematics"),
    count(col("DSA") >= 40).alias("pass_dsa")
).collect()[0]

# Display basic analysis results
print("Basic Analysis Results:")
print("=======================")
print(f"Total Students: {pass_fail_stats['total_students']}")

print("\nAverage Marks:")
for subject in ["Electronics", "Programming", "Database", "Data_Science", "Mathematics", "DSA"]:
    key = f"avg_{subject.lower().replace(' ', '_')}"
    print(f"  {subject}: {analysis[key]:.2f}")

print("\nMinimum Marks:")
for subject in ["Electronics", "Programming", "Database", "Data_Science", "Mathematics", "DSA"]:
    key = f"min_{subject.lower().replace(' ', '_')}"
    print(f"  {subject}: {analysis[key]}")

print("\nMaximum Marks:")
for subject in ["Electronics", "Programming", "Database", "Data_Science", "Mathematics", "DSA"]:
    key = f"max_{subject.lower().replace(' ', '_')}"
    print(f"  {subject}: {analysis[key]}")

print("\nStandard Deviation:")
for subject in ["Electronics", "Programming", "Database", "Data_Science", "Mathematics", "DSA"]:
    key = f"std_{subject.lower().replace(' ', '_')}"
    print(f"  {subject}: {analysis[key]:.2f}")

print("\nPass/Fail Statistics:")
for subject in ["Electronics", "Programming", "Database", "Data_Science", "Mathematics", "DSA"]:
    key = f"pass_{subject.lower().replace(' ', '_')}"
    pass_count = pass_fail_stats[key]
    fail_count = pass_fail_stats['total_students'] - pass_count
    pass_rate = pass_count / pass_fail_stats['total_students'] * 100
    fail_rate = fail_count / pass_fail_stats['total_students'] * 100
    print(f"  {subject}:")
    print(f"    Passed: {pass_count} ({pass_rate:.2f}%)")
    print(f"    Failed: {fail_count} ({fail_rate:.2f}%)")

# Display Average Total Marks and Pass/Fail Status
print("\nAverage Total Marks and Pass/Fail Status:")
df.select("Student_ID", "Name", "Department",
          "Electronics_Status", "Programming_Status", "Database_Status",
          "Data_Science_Status", "Mathematics_Status", "DSA_Status",
          "Average_Total_Marks").show(10, truncate=False)

# Display Top 10 Performers
print("\n--- Top 10 Performers ---")
df.orderBy(col("Average_Total_Marks").desc()).select("Student_ID", "Name", "Department", "Average_Total_Marks").show(10, truncate=False)

# Display Subject-wise Statistics
print("\n--- Subject-wise Statistics ---")
subject_stats = df.select(
    "Electronics", "Programming", "Database", "Data_Science", "Mathematics", "DSA"
).summary("count", "mean", "stddev", "min", "max").toPandas()

for subject in ["Electronics", "Programming", "Database", "Data_Science", "Mathematics", "DSA"]:
    print(f"\n{subject}:")
    print(subject_stats[subject])

# Create a comprehensive dashboard with various chart types
fig = make_subplots(rows=4, cols=2, subplot_titles=(
    "Average Marks (Bar)", "Average Marks (Line)",
    "Minimum Marks", "Maximum Marks",
    "Standard Deviation", "Pass Rate",
    "Fail Rate", "Overall Pass/Fail Distribution"
),
    specs=[[{'type':'bar'}, {'type':'scatter'}],
           [{'type':'bar'}, {'type':'bar'}],
           [{'type':'bar'}, {'type':'bar'}],
           [{'type':'bar'}, {'type':'pie'}]],
    vertical_spacing=0.08,
    horizontal_spacing=0.07
)

# Average Marks (Bar)
fig.add_trace(go.Bar(x=["Electronics", "Programming", "Database", "Data Science", "Mathematics", "DSA"],
                     y=[analysis["avg_electronics"], analysis["avg_programming"], analysis["avg_database"],
                        analysis["avg_data_science"], analysis["avg_mathematics"], analysis["avg_dsa"]],
                     name="Average Marks"), row=1, col=1)

# Average Marks (Line)
fig.add_trace(go.Scatter(x=["Electronics", "Programming", "Database", "Data Science", "Mathematics", "DSA"],
                         y=[analysis["avg_electronics"], analysis["avg_programming"], analysis["avg_database"],
                            analysis["avg_data_science"], analysis["avg_mathematics"], analysis["avg_dsa"]],
                         mode='lines+markers',
                         name="Average Marks"), row=1, col=2)

# Minimum Marks
fig.add_trace(go.Bar(x=["Electronics", "Programming", "Database", "Data Science", "Mathematics", "DSA"],
                     y=[analysis["min_electronics"], analysis["min_programming"], analysis["min_database"],
                        analysis["min_data_science"], analysis["min_mathematics"], analysis["min_dsa"]],
                     name="Minimum Marks"), row=2, col=1)

# Maximum Marks
fig.add_trace(go.Bar(x=["Electronics", "Programming", "Database", "Data Science", "Mathematics", "DSA"],
                     y=[analysis["max_electronics"], analysis["max_programming"], analysis["max_database"],
                        analysis["max_data_science"], analysis["max_mathematics"], analysis["max_dsa"]],
                     name="Maximum Marks"), row=2, col=2)

# Standard Deviation
fig.add_trace(go.Bar(x=["Electronics", "Programming", "Database", "Data Science", "Mathematics", "DSA"],
                     y=[analysis["std_electronics"], analysis["std_programming"], analysis["std_database"],
                        analysis["std_data_science"], analysis["std_mathematics"], analysis["std_dsa"]],
                     name="Standard Deviation"), row=3, col=1)

# Pass Rate
pass_rates = [pass_fail_stats["pass_electronics"] / pass_fail_stats["total_students"],
              pass_fail_stats["pass_programming"] / pass_fail_stats["total_students"],
              pass_fail_stats["pass_database"] / pass_fail_stats["total_students"],
              pass_fail_stats["pass_data_science"] / pass_fail_stats["total_students"],
              pass_fail_stats["pass_mathematics"] / pass_fail_stats["total_students"],
              pass_fail_stats["pass_dsa"] / pass_fail_stats["total_students"]]

fig.add_trace(go.Bar(x=["Electronics", "Programming", "Database", "Data Science", "Mathematics", "DSA"],
                     y=pass_rates,
                     name="Pass Rate"), row=3, col=2)

# Fail Rate
fail_rates = [1 - rate for rate in pass_rates]

fig.add_trace(go.Bar(x=["Electronics", "Programming", "Database", "Data Science", "Mathematics", "DSA"],
                     y=fail_rates,
                     name="Fail Rate"), row=4, col=1)

# Overall Pass/Fail Distribution
total_pass = sum(pass_fail_stats[f"pass_{subject.lower().replace(' ', '_')}"] for subject in ["Electronics", "Programming", "Database", "Data_Science", "Mathematics", "DSA"])
total_fail = pass_fail_stats['total_students'] * 6 - total_pass

fig.add_trace(go.Pie(labels=['Passed', 'Failed'],
                     values=[total_pass, total_fail],
                     name="Overall Pass/Fail"), row=4, col=2)

# Update layout
fig.update_layout(height=1600, width=1200, title_text="Result Management System Dashboard")

# Display the dashboard
fig.show()

# Stop the Spark session
spark.stop()

Basic Analysis Results:
Total Students: 10000

Average Marks:
  Electronics: 51.54
  Programming: 58.08
  Database: 45.07
  Data_Science: 41.54
  Mathematics: 38.85
  DSA: 54.10

Minimum Marks:
  Electronics: 0
  Programming: 0
  Database: 0
  Data_Science: 0
  Mathematics: 0
  DSA: 0

Maximum Marks:
  Electronics: 90
  Programming: 95
  Database: 85
  Data_Science: 80
  Mathematics: 75
  DSA: 92

Standard Deviation:
  Electronics: 24.99
  Programming: 24.81
  Database: 24.58
  Data_Science: 23.22
  Mathematics: 21.79
  DSA: 25.04

Pass/Fail Statistics:
  Electronics:
    Passed: 10000 (100.00%)
    Failed: 0 (0.00%)
  Programming:
    Passed: 10000 (100.00%)
    Failed: 0 (0.00%)
  Database:
    Passed: 10000 (100.00%)
    Failed: 0 (0.00%)
  Data_Science:
    Passed: 10000 (100.00%)
    Failed: 0 (0.00%)
  Mathematics:
    Passed: 10000 (100.00%)
    Failed: 0 (0.00%)
  DSA:
    Passed: 10000 (100.00%)
    Failed: 0 (0.00%)

Average Total Marks and Pass/Fail Status:
+----------+-----